In [5]:
import requests
import pandas as pd
import json
import os
from datetime import date, time, datetime
import ipywidgets as widgets
import numpy as np
from IPython.display import display

from utils import *
from logifleet_requests import *

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as font_manager
import matplotlib

from io import BytesIO
import base64

POPPINS_REGULAR_PATH = "static\Poppins-Regular.ttf"
font_manager.fontManager.addfont(POPPINS_REGULAR_PATH)
POPPINS_REGULAR = font_manager.FontProperties(fname=POPPINS_REGULAR_PATH)
matplotlib.rcParams["font.family"] = POPPINS_REGULAR.get_name()
matplotlib.rcParams["figure.edgecolor"] = "#BCCAD4"
sns.set_style(style="white")

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [6]:
def run(startDate, endDate, jwt):
    
    # VEHICLES & GROUPS
    vehicles_df, groups_df = get_vehicles_and_groups(jwt)
    
    # REPORTS
    reports_df = get_reports(startDate, endDate, jwt)
    
    # TRIPS
    trips_df = get_trips(startDate, endDate, jwt)
    
    # une fois qu'on a les trips, il faut lier trips_df et vehicles_df avec leur asset_reference (vehicles_df -> reference)
    # comme ça on peut lier la distance (dans trips_df) et la conso du vehicule (type de veh)
    # il y a une colonne qui s'appelle tripDistanceOnDistanceUnit et une DistanceUnit mais elle est tt le temps None
    
    vehicleCategoryName_dict = dict(zip(vehicles_df['reference'], vehicles_df['vehicleCategoryName']))
    engineType_dict = dict(zip(vehicles_df['reference'], vehicles_df['engineType']))
    vehicleGroup_dict = dict(zip(vehicles_df['reference'], vehicles_df['groupId']))
    groupName_dict = dict(zip(groups_df['id'], groups_df['name']))
    
    trips_df['vehicleCategoryName'] = trips_df['asset_reference'].map(vehicleCategoryName_dict)
    trips_df['engineType'] = trips_df['asset_reference'].map(engineType_dict)
    trips_df['groupId'] = trips_df['asset_reference'].map(vehicleGroup_dict)
    trips_df['groupName'] = trips_df['groupId'].map(groupName_dict)
    
    trips_df['co2_facteur_wtt'] = trips_df.apply(column_co2_facteur_wtt, axis=1)
    trips_df['masse_carburant'] = trips_df.apply(column_masse_carburant, axis=1)
    trips_df['unite_carburant'] = trips_df.apply(column_unite_carburant, axis=1)
    trips_df['co2_direct'] = trips_df.apply(calcul_co2_direct, axis=1)
    trips_df['co2_eq_kg'] = trips_df.apply(calcul_co2_eq_kg, axis=1)
    
    # trips_df['distance'].mean() # trajets de 4.3 km en moyenne
    # trips_df['co2_eq_kg'].sum() / trips_df['distance'].sum() # environ 199 g émis par km en moyenne
    
    # formattage dates : 
    trips_df['startingDate_str'] = trips_df.apply(extract_year_month_name, axis = 1)
    trips_df['month'] = trips_df.apply(extract_year_month_number, axis = 1)
    
    trips_df = trips_df.reset_index().rename(columns = {
        'month' : 'Mois', 
         'engineType' : 'Type de moteur', 
         'groupName' : 'Groupe', 
         'unite_carburant' : 'Unité carburant', 
         'distance' : 'Distance [km]', 
         'co2_direct' : 'CO2 direct', 
         'co2_facteur_wtt' : 'CO2-Facteur WTT',  
         'co2_eq_kg' : 'CO2 équivalent [kg]', 
         'masse_carburant' : 'Masse carburant',
    })
    
    trips_per_month = trips_df[['Distance [km]', 
                                  'CO2 direct', 
                                   'CO2-Facteur WTT',
                                  'CO2 équivalent [kg]',
                                   'Groupe',  
                                   'Type de moteur',  
                                'Masse carburant',
                                  'startingDate_str', 
                                  'Mois']].groupby(by = ['Mois', 'startingDate_str', 'Groupe', 'Type de moteur']).sum()
    
    scores_df = get_scores(startDate, endDate, jwt, trips_df)
    
    # formattage dates : 
    
    scores_df["startingDate"] = scores_df["timeDimension"].apply(lambda x : x + "-01T") # pour pouvoir formatter les dates et grouper par mois et merge avec trips_df
    scores_df['startingDate_str'] = scores_df.apply(extract_year_month_name, axis = 1)
    scores_df['Mois'] = scores_df.apply(extract_year_month_number, axis = 1)
    
    scores_df['Type de moteur'] = scores_df.reset_index()['vehicle_name'].map(engineType_dict)
    
    
    scores_df = scores_df.reset_index().rename(columns = {'group_name' : 'Groupe'})
    
    scores_per_group_and_month_mean = scores_df[['Groupe', 
                                            'globalScore', 
                                            'Mois', 
                                            'startingDate_str', 
                                            'Type de moteur']].groupby(['Mois', 'startingDate_str', 'Groupe', 'Type de moteur']).mean()
    
    scores_per_group_and_month_sum = scores_df[['Groupe', 
                                            'numberOfTrips', 
                                            'Mois', 
                                            'startingDate_str', 
                                            'Type de moteur']].groupby(['Mois', 'startingDate_str', 'Groupe', 'Type de moteur']).sum()
    
    scores_per_group_and_month = pd.concat([scores_per_group_and_month_mean, scores_per_group_and_month_sum], axis = 1)
    
    # CONCAT LES 2 DF
    
    df_per_month = pd.concat([trips_per_month, scores_per_group_and_month], axis = 1).reset_index()
    
    dict_unite_carburant = {
        "DIESEL" : "L", 
        "ELECTRIC" : "kMh", 
        "GAS" : "Kg", 
        "GASOLINE" : "L", 
        "HYBRID" : "L"
    }
    
    df_per_month['Unité carburant'] = df_per_month['Type de moteur'].map(dict_unite_carburant)
    
    df_total = df_per_month.groupby('Groupe').sum().reset_index()
    df_total.rename(columns = {'globalScore' : 'Eco score', 'numberOfTrips' : 'Nombre de trajets'}, inplace = True)
    df_total = df_total[['Groupe', 'Distance [km]', 'CO2 équivalent [kg]', 'Eco score', 'Nombre de trajets']]

    print("Terminé!")
    return df_total


# GRAPHIQUES

def bar_plot(df, column, formatted_startDate, formatted_endDate):
    plt.figure(figsize=(12, 6))
    sns.barplot(
        data=df,
        x='Groupe',
        y=column,
    )
    plt.title("%s - total par groupe du %s au %s" % (column, formatted_startDate, formatted_endDate))
    plt.grid(which="major")
    plt.xticks(rotation=45, ha='right')
    plt.xlabel('')
    sns.despine()
    plt.show()





In [7]:
def on_run_clicked(b, startDate_date, endDate_date, jwt):
    with output:

        start_dt = datetime.combine(startDate_date, time.min)
        startDate = start_dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
        
        end_dt = datetime.combine(endDate_date, time.min)
        endDate = end_dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
        
        print(startDate, endDate)

        df_total = run(startDate, endDate, jwt)

        print(df_total)
        
        date_startDate = datetime.strptime(startDate, '%Y-%m-%dT%H:%M:%S.%fZ')
        formatted_startDate = date_startDate.strftime('%d/%m/%Y')
        
        date_endDate = datetime.strptime(endDate, '%Y-%m-%dT%H:%M:%S.%fZ')
        formatted_endDate = date_endDate.strftime('%d/%m/%Y')

        bar_plot(df_total, "Distance [km]", formatted_startDate, formatted_endDate)
        bar_plot(df_total, "CO2 équivalent [kg]", formatted_startDate, formatted_endDate)
        bar_plot(df_total, "Eco score", formatted_startDate, formatted_endDate)
        bar_plot(df_total, "Nombre de trajets", formatted_startDate, formatted_endDate)

        # Bouton export

        def export_to_excel(b):
            buffer = BytesIO()
            df_total.to_excel(buffer, index=False)
            buffer.seek(0)
            
            b64 = base64.b64encode(buffer.read()).decode()
            href = f'<a download="export.xlsx" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" target="_blank">📁 Télécharger le fichier Excel</a>'
            
            download_link.value = href
    
        export_button = widgets.Button(description="Exporter en Excel", button_style='success')
        export_button.on_click(export_to_excel)
        
        # HTML pour le lien de téléchargement
        download_link = widgets.HTML(value="")
        
        # Affichage
        display(export_button, download_link)

In [9]:
username_input = widgets.Text(
    placeholder='Nom d’utilisateur',
    description='Utilisateur:',
    disabled=False
)

password_input = widgets.Password(
    placeholder='Mot de passe',
    description='Mot de passe:',
    disabled=False
)

login_button = widgets.Button(
    description='Se connecter',
    button_style='success'
)

output = widgets.Output()

def check_login(b):
    with output:

        username = username_input.value
        password = password_input.value

        payload = {
            "username": username,
            "password": password,
        }
        headers = {
            "Content-Type": "application/json"
        }
        
        response = requests.post(url, json=payload, headers=headers)
        jwt = response.json().get('jwt')

        if jwt:
            print("✅ Connexion réussie.")
            show_main_app(jwt)  # Appel de la fonction principale
        else:
            print("❌ Identifiants incorrects.")

def show_main_app(jwt):
    
    # Widgets pour la sélection de la période
    start_date_picker = widgets.DatePicker(description='Date de début')
    end_date_picker = widgets.DatePicker(description='Date de fin')
    
    def on_date_change(change):
        print("Date de début :", start_date_picker.value)
        print("Date de fin   :", end_date_picker.value)
    
    start_date_picker.observe(on_date_change, names='value')
    end_date_picker.observe(on_date_change, names='value')
    
    widgets_box = widgets.HBox([start_date_picker, end_date_picker])
    display(widgets_box)

    if not start_date_picker.value or not end_date_picker.value:
        print("❌ Merci de sélectionner les deux dates.")
        return
    if start_date_picker.value > end_date_picker.value:
        print("⚠️ La date de début doit précéder la date de fin.")
        return
    
    print(f"✅ Période valide : du {start_date_picker.value} au {end_date_picker.value}")

    startDate_date = start_date_picker.value
    endDate_date = end_date_picker.value

    run_button = widgets.Button(description="Afficher", button_style='primary')
    output = widgets.Output()
    
    run_button.on_click(lambda b: on_run_clicked(b, start_date_picker.value, end_date_picker.value, jwt))
    
    display(widgets.VBox([run_button, output]))
    
    start_dt = datetime.combine(startDate_date, time.min)
    startDate = start_dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    
    end_dt = datetime.combine(endDate_date, time.min)
    endDate = end_dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
    
    print(startDate, endDate)

    df_total = run(startDate, endDate, jwt)

    print(df_total)
    
    date_startDate = datetime.strptime(startDate, '%Y-%m-%dT%H:%M:%S.%fZ')
    formatted_startDate = date_startDate.strftime('%d/%m/%Y')
    
    date_endDate = datetime.strptime(endDate, '%Y-%m-%dT%H:%M:%S.%fZ')
    formatted_endDate = date_endDate.strftime('%d/%m/%Y')

    bar_plot(df_total, "Distance [km]", formatted_startDate, formatted_endDate)
    bar_plot(df_total, "CO2 équivalent [kg]", formatted_startDate, formatted_endDate)
    bar_plot(df_total, "Eco score", formatted_startDate, formatted_endDate)
    bar_plot(df_total, "Nombre de trajets", formatted_startDate, formatted_endDate)

    # Bouton export

    def export_to_excel(b):
        buffer = BytesIO()
        df_total.to_excel(buffer, index=False)
        buffer.seek(0)
        
        b64 = base64.b64encode(buffer.read()).decode()
        href = f'<a download="export.xlsx" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" target="_blank">📁 Télécharger le fichier Excel</a>'
        
        download_link.value = href

    export_button = widgets.Button(description="Exporter en Excel", button_style='success')
    export_button.on_click(export_to_excel)
    
    # HTML pour le lien de téléchargement
    download_link = widgets.HTML(value="")
    
    # Affichage
    display(export_button, download_link)





login_button.on_click(check_login)

# Affichage du formulaire
display(username_input, password_input, login_button, output)


Text(value='', description='Utilisateur:', placeholder='Nom d’utilisateur')

Password(description='Mot de passe:', placeholder='Mot de passe')

Button(button_style='success', description='Se connecter', style=ButtonStyle())

Output()